In [7]:
# Import necessary packages
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.compose import ColumnTransformer


In [8]:
# set_config(transform_output="default")

In [9]:
df = pd.read_csv("tj_pitching_merge.csv")
# print(df["pitches"].value_counts())
print(df.shape)
df.head()

(3277, 3)


,pitches,Player,No. TJ Surgeries
0,52108.0,Justin Verlander,1.0
1,48905.0,Max Scherzer,0.0
2,47573.0,Zack Greinke,0.0
3,44996.0,Jon Lester,0.0
4,44494.0,Clayton Kershaw,0.0


In [10]:
# Train Test Split
X = df['pitches']
y = df['No. TJ Surgeries']

In [15]:
X = X.to_frame()
y = y.to_frame()
X

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [16]:
X.info

<bound method DataFrame.info of       pitches
0     52108.0
1     48905.0
2     47573.0
3     44996.0
4     44494.0
...       ...
3272      3.0
3273      3.0
3274      3.0
3275      3.0
3276      2.0

[3277 rows x 1 columns]>

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [18]:

print(X_train.info)
print(X_train.shape)
print(X_test.shape)

<bound method DataFrame.info of       pitches
723    4943.0
3021     31.0
471    7370.0
3048     28.0
2262    436.0
...       ...
1972    739.0
934    3633.0
3174     14.0
3049     28.0
978    3354.0

[2621 rows x 1 columns]>
(2621, 1)
(656, 1)


In [20]:
clf1 = RandomForestClassifier(random_state=42)
clf2 = SVC(probability=True, random_state=42)
clf3 = LogisticRegression(random_state=42)
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
clf7 = GradientBoostingClassifier(random_state=42)


In [36]:
# Initiaze the hyperparameters for each dictionary
param1 = {}
param1['classifier__n_estimators'] = [10, 50, 100, 250]
param1['classifier__max_depth'] = [1, 10, 20]
param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param3 = {}
param3['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param3['classifier__penalty'] = ['l1', 'l2']
param3['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param3['classifier'] = [clf3]

In [42]:
pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3]
#, param2, param3

In [43]:
# Train the grid search model
gs = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring='roc_auc').fit(X_train, y_train)

c:\conda\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\conda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
100 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\conda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\conda\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

In [44]:
# Best performing model and its corresponding hyperparameters
best = gs.best_params_

# ROC-AUC score for the best model
score = gs.best_score_

In [45]:
best

{'classifier': RandomForestClassifier(random_state=42),
 'classifier__class_weight': None,
 'classifier__max_depth': 1,
 'classifier__n_estimators': 10}

In [46]:
score

nan